In [1]:
import cytoflow as flow
import matplotlib.pyplot as plt
import seaborn as sns
import math

%matplotlib inline

import glob
import numpy as np
import pandas as pd
import matplotlib.colors as colors
from scipy import optimize
from scipy.optimize import curve_fit, minimize



darks=['#3a80b0', '#ff7200', '#984ea3', '#7fc97f',  '#e261b0']
pastels=['#b3cde3','#fed9a6','#decbe4','#ccebc5',  '#f4cae4']
superdarks=['#1f5a82', '#ee4e00', '#771e85', '#41ab5d', '#cf3092']



plt.rcParams['xtick.labelsize']=20
plt.rcParams['ytick.labelsize']=20

In [2]:
def import_fcs_files(path, sample_list, names_in):
    files=glob.glob(path)
    print(len(files))   
    
    samples=[]
    for f in files: 
        well=f[-7:-4]
        #print(well)|
        name=sample_list[sample_list['well']==well]['sample'].reset_index(drop=True)[0]
        #print(name)
        if name in names_in:
            #print(name)
            sample=flow.Tube(file=f, conditions={'name':name})
            samples.append(sample) 
    expout=flow.ImportOp(tubes=samples, conditions={'name':'category'}).apply()
    return(expout)

def plot_fsc_ssc(df, ax, title):
    
    xbins=10**np.linspace(2,5.5,1000)
    ybins=10**np.linspace(2,5.5,1000)
    
    counts, _, _=np.histogram2d(df['FSC-H'], df['SSC-H'], bins=(xbins, ybins))
    
    ax.pcolormesh(xbins, ybins, counts.T, norm=colors.LogNorm(.000001, vmax=counts.max()), cmap='viridis')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('FSC-H', fontsize='xx-large')
    ax.set_ylabel('SSC-H', fontsize='xx-large')
    ax.set_title(title)


def plot_rect(pt1, pt2, pt3, pt4, ax):
    ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], '-r')
    ax.plot([pt1[0], pt4[0]], [pt1[1], pt4[1]], '-r')
    ax.plot([pt2[0], pt3[0]], [pt2[1], pt3[1]], '-r')
    ax.plot([pt3[0], pt4[0]], [pt3[1], pt4[1]], '-r')

    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_xlim(10**2, 10**5.5)
    ax.set_ylim(10**2, 10**5.5)
    
def gen_pts(x_low, x_high, y_low, y_high):
    pt1=[x_low, y_low]
    pt2=[x_low, y_high]
    pt3=[x_high, y_high]
    pt4=[x_high, y_low]
    return(pt1, pt2, pt3, pt4)

def hist_params(fig, ax, title, fname):
    
    ax.grid(False)
    ax.set_xscale('symlog')
    #ax.set_yscale('log')
#     ax.set_title(title, fontsize=56)

#     plt.xlabel('GFP fluorescence (a.u.)', fontsize=48)
#     plt.ylabel('counts', fontsize=48)


    #ax.set_yticks([0, 10, 20, 30], fontsize=60) 
    ax.set_yticks([0, 10000, 20000])
    ax.set_yticklabels(['0', '10', '20'])
    ax.set_ylim(0, 20000)
    ax.set_xlim(1e1,5e4)
    ax.set_ylabel(u'Counts (x10\u00B3)', fontsize=40)
    ax.set_xlabel('Fluorescence (GFP, au)', fontsize=40)

    ax.tick_params(which='major', direction='out', length=15, width=4)
    ax.tick_params(which='minor', direction='out', length=0, width=2)
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(4)
        ax.spines[axis].set_color('black')
    ax.tick_params(axis='both', which='major', labelsize=40)

    plt.show()
    plt.tight_layout()
#     fig.savefig(fname+'.pdf', bbox_inches='tight')
    plt.close()

    

nbins=10**np.linspace(-1, 5.1, 300)

In [5]:
floc1='flow_fig5d_rep1'
floc2='flow_fig5d_rep2'
floc3='flow_fig5d_rep3'

samp1=pd.read_csv(floc1+r'\sample_list.csv', names=['well', 'sample'])
samp2=pd.read_csv(floc2+r'\plate_1\sample_list.csv', names=['well', 'sample'])
samp3=pd.read_csv(floc3+r'\plate_1\sample_list.csv', names=['well', 'sample'])

names1=[name for name in samp1['sample'].unique() if name!='blank']
names2=[name for name in samp2['sample'].unique() if name!='blank']
names3=[name for name in samp3['sample'].unique() if name!='blank']


exp1=import_fcs_files(floc1+r'\*.fcs', 
                   samp1, names1)
exp2=import_fcs_files(floc2+r'\plate_1\*.fcs', samp2, names2)
exp3=import_fcs_files(floc3+r'\plate_1\*.fcs', samp3, names3)

exp1.data.head()

28
28
26


,FITC-A,FSC-A,FSC-H,FSC-W,PE-Texas Red-A,Pacific Blue-A,SSC-A,SSC-H,SSC-W,Time,name
0,286.899994,73.599998,596.0,8093.035645,41.310001,258.500000,28095.298828,24460.0,75276.109375,54.500000,all_neg
1,41.799999,-380.000000,288.0,0.000000,95.580002,-234.300003,10782.500000,11220.0,62980.566406,54.500000,all_neg
2,157.699997,-360.000000,233.0,0.000000,30.780001,207.900009,16743.750000,16228.0,67618.835938,54.599998,all_neg
3,270.750000,-423.200012,321.0,0.000000,-119.070000,-1413.500000,12659.700195,12941.0,64111.437500,54.599998,all_neg
4,19.949999,-292.800018,269.0,0.000000,75.330002,394.899994,11932.950195,12093.0,64668.640625,54.599998,all_neg


In [7]:
fits_final=pd.read_csv('fits_fig5d.csv')
fits_final.head()

fits_final['fa_mean']=fits_final[['fa1', 'fa2', 'fa3']].mean(axis=1)
fits_final

#samples shown in fig 5d are 'POS'

,sample,r1_mu1,r1_s1,r1_A1,r1_mu2,r1_s2,r1_A2,r1_fneg,A1+A2,Aneg,...,r3_A1,r3_mu2,r3_s2,r3_A2,r3_fneg,A1+A2.2,Aneg.2,Atot.2,fa3,fa_mean
0,NEG,5.370937,0.521387,20443.80131,8.629031,0.026040,10.176935,0.045853,20453.97825,982.956276,...,22503.15595,8.828724,0.154651,2.907210,0.035289,22506.06316,823.263099,23329.32626,0.000125,0.000226
1,POS,5.368806,0.551035,20259.39929,8.152864,0.514444,841.109023,0.049398,21100.50831,1096.489866,...,21064.17093,8.407970,0.556865,1161.192447,0.036988,22225.36338,853.657906,23079.02129,0.050314,0.044573
2,A_off,5.812228,0.489481,21480.45259,8.073656,0.092017,0.895658,0.019761,21481.34825,433.060091,...,24267.86534,8.694241,0.150556,0.277965,0.007500,24268.14331,183.392425,24451.53573,0.000011,0.000025
3,C_off,5.876775,0.476466,21694.90678,7.388766,0.053152,2.141898,0.012878,21697.04868,283.049106,...,24061.88369,8.636397,0.203435,1.023333,0.008386,24062.90702,203.490620,24266.39764,0.000042,0.000055
4,D_off,5.575518,0.502337,20310.14752,9.200095,0.360137,14.124561,0.041594,20324.27209,882.063441,...,23003.55119,8.764185,0.036582,0.495898,0.025158,23004.04709,593.661405,23597.70850,0.000021,0.000276
5,E_off,5.645880,0.501677,20555.20903,8.174192,0.016044,0.191028,0.040068,20555.40006,857.991535,...,16676.93907,8.781317,0.026867,0.640754,0.019639,16677.57983,334.094788,17011.67462,0.000038,0.000037
6,F_off,5.852092,0.482910,21722.07180,8.449819,0.226008,1.127112,0.016890,21723.19891,373.211191,...,23936.29918,8.464045,0.017237,0.253453,0.009069,23936.55264,219.064434,24155.61707,0.000010,0.000053
7,A_on,5.732503,0.530451,19315.94825,8.220076,0.495778,4115.362888,0.018845,23431.31114,450.038630,...,21950.17323,8.416135,0.450454,2399.668107,0.009386,24349.84134,230.709015,24580.55035,0.097625,0.165815
8,C_on,5.739689,0.545404,15770.10473,8.206363,0.524829,7967.070373,0.014642,23737.17510,352.724008,...,23055.34785,8.327241,0.380419,1089.146607,0.010865,24144.49445,265.223541,24409.71799,0.044619,0.138403
9,D_on,5.511851,0.589736,20538.13200,7.390521,0.355487,1859.104421,0.043806,22397.23642,1026.091333,...,21387.72790,7.901335,0.353429,1189.458414,0.032630,22577.18631,761.551594,23338.73790,0.050965,0.051300
